In [ ]:
import torch
data = torch.randn([1, 25088])

In [ ]:
x1 = torch.concat([data, torch.zeros([1, 512])], dim=1)
x2 = data

y1 = torch.stft(x1, 512, hop_length=256, return_complex=True)
y2 = torch.stft(x2, 512, hop_length=256, return_complex=True)
y_ = torch.stft(torch.concat([x2[:, 24832:], torch.zeros([1, 256])], dim=1), 512, hop_length=256, return_complex=True, center=False)
y2 = torch.concat([y2[:,:, :-1], y_, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]])], dim=2)
# y2 = torch.concat([y2, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]])], dim=2)

In [ ]:
print(torch.sum(torch.abs(torch.abs(y1[:,:, 99]) - torch.abs(y2[:,:, 99]))))

In [ ]:
print(torch.sum(torch.abs(torch.abs(y1) - torch.abs(y2))))

In [ ]:
x1 = torch.concat([data, torch.zeros([1, 512])], dim=1)
x2 = data

y1 = torch.stft(x1, 512, hop_length=256, return_complex=True, center=False)
y2 = torch.stft(x2, 512, hop_length=256, return_complex=True, center=False)
y_ = torch.stft(torch.concat([x2[:, 24832:], torch.zeros([1, 256])], dim=1), 512, hop_length=256, return_complex=True, center=False)
y2 = torch.concat([y2, y_, torch.zeros([1, 257, y1.shape[2]-y2.shape[2]-1])], dim=2)
print(torch.sum(torch.abs(torch.abs(y1) - torch.abs(y2))))

In [ ]:
import time
from progressbar import ProgressBar, Percentage, Bar, Timer, ETA

widgets = ['Progress: ',Percentage(), ' ', Bar('#'),' ', Timer(),
           ' ', ETA()]
progress = ProgressBar(widgets=widgets).start()
for i in progress(range(100)):
    time.sleep(0.04)

In [ ]:
with open("wav_polqa.list", "r", encoding="utf-8") as f:
    li = f.read().splitlines()

import random
random.shuffle(li)

li_ = li[:60000]
with open("wav_polqa_big.list", "w", encoding="utf-8") as f:
    for l in li_:
        f.write(l+"\n")


In [ ]:
from trainer_utils import EarlyStopping, Args
import time
x = 10
args = Args(model_type="")
print(args.patience)
early_stopping = EarlyStopping(args.patience, args.delta_loss)
for i in range(20):
    time.sleep(0.1)
    if i < 5:
        x -= 0.1
    else:
        x += 1e-4
    print(x)
    if early_stopping(x):
        print("Early stopping")
        break

In [ ]:
with open("wav_polqa.list", "r", encoding="utf-8") as f:
    li = f.read().splitlines()

polqa = [float(l.split(",")[1]) for l in li]


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['font.sans-serif'] = ['Simhei']  # 显示中文
plt.rcParams['axes.unicode_minus'] = False  # 显示负号
plt.rcParams['font.size'] = "14.0"
dpi = 300
polqa = np.array(polqa)
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']  # 每一块的颜色
part = [len(polqa[(polqa>=i) & (polqa < (i + 1))]) for i in range(1,5)]
plt.figure(dpi=100)
plt.pie(part, labels=["1-2", "2-3", "3-4", "4-5"], autopct='%1.1f%%', colors=colors, startangle=90)
plt.title("MOS分数分布")
plt.tight_layout()
plt.savefig("images/MOS分数分布图.png", dpi=dpi)

In [ ]:
with open("wav_polqa.list", "r", encoding="utf-8") as f:
    li = f.read().splitlines()

polqa = [float(l.split(",")[1]) for l in li]

In [ ]:
import numpy as np
step = 20
dis_num = np.zeros([int(400 / step)])
for p in polqa:
    idx = (int(p * 100) - 100) // step
    dis_num[idx] += 1
print(dis_num)

In [ ]:
dis_prop = max(dis_num) / dis_num
np.sqrt(dis_prop)

In [ ]:
from trainer_utils import plot_metric
import numpy as np
path = r"D:\work\speechEnhancement\speechQuality\QualityNetPOLQA\results\dpcrn_qsehasa20240518_134721\data\train_metric.npy"
data = np.load(path, allow_pickle=True).item()
plot_metric({"train_loss": data['train_loss'], "valid_loss": data["valid_loss"]},
                                  title="train and valid loss")

In [ ]:
import torch

model = torch.load(r"D:\work\speechEnhancement\speechQuality\QualityNetPOLQA\models\dpcrn_qsehasa20240518_134721\best.pt")
model.eval()


In [22]:
from utils import seed_everything
import torch
import random
import numpy as np

seed_everything(42)

In [24]:
print(torch.randn((4, 8)))
print(np.random.rand(4, 8))

In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
x = np.random.rand(10)

class dataset(Dataset):
    def __init__(self, x):
        self.x = x
    def __getitem__(self, idx):
        return self.x[idx]
    
    def __len__(self):
        return len(self.x)

In [16]:
d = dataset(x=x)
l = DataLoader(d, batch_size=4, shuffle=True)
i_l = iter(l)
for i in range(20):
    try:
        y = next(i_l)
    except StopIteration:
        i_l = iter(l)
        y = next(i_l)
    print(y)

In [ ]:
from utils import get_logging

logging = get_logging("log.txt")
logging.info("hello")
logging.info("hello")
logging.info("hello")